# Paths

In [1]:
path = '/home/tih_isi_7/G-NeuroDAVIS/'
res_plots = path + 'Results/Plots/'
path_emb = path + 'Data/Embeddings/'
path_tab = path + 'Results/Tables/'

# Importing libraries

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import sys
sys.path.append(path)

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import keras
import ipynbname
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from utils import Datasets as ds
from keras.models import load_model
from utils import DownstreamAnalysis as DA

dataname = ipynbname.name().rsplit("_")[0]
txt = %pwd
methodname = txt.rsplit("/")[-1]

# Load labels and embeddings

In [3]:
y, _ = ds.LoadData(dataname, DType='None', LType=True)
x_embedding = np.array(pd.read_csv(path_emb+dataname+'/'+dataname+'_'+methodname+'.csv', index_col=0, header=0))

# Downstream analysis

### k-NN

In [4]:
neighbour = [5, 15, 25, 35, 45]
Score_knn = []
for n in neighbour:
    Score_knn.append(DA.Knn(x_embedding, y, n))
Score_knn = pd.DataFrame(Score_knn, index=['neighbour '+ str(s) for s in neighbour],
                         columns = ['Accuracy', 'Precision','Recall','F1-Score'])
Score_knn.to_csv(path_tab+dataname+'_'+methodname+'_knn.csv')
Score_knn

,Accuracy,Precision,Recall,F1-Score
neighbour 5,0.703083,0.700148,0.703083,0.699187
neighbour 15,0.724000,0.721295,0.724000,0.720819
neighbour 25,0.727917,0.724283,0.727917,0.724078
neighbour 35,0.729583,0.725358,0.729583,0.725210
neighbour 45,0.728333,0.723377,0.728333,0.723540


### Random Forest

In [5]:
n_tree = [20,40,60,80,100]
Score_rfc = []
for n in n_tree:
    Score_rfc.append(DA.RFC(x_embedding, y, n))
Score_rfc = pd.DataFrame(Score_rfc, index=['n_tree '+ str(s) for s in n_tree],
                         columns = ['Accuracy', 'Precision','Recall','F1-Score'])
Score_rfc.to_csv(path_tab+dataname+'_'+methodname+'_rfc.csv')
Score_rfc

,Accuracy,Precision,Recall,F1-Score
n_tree 20,0.693083,0.690373,0.693083,0.691026
n_tree 40,0.693667,0.691443,0.693667,0.692045
n_tree 60,0.694167,0.691738,0.694167,0.692455
n_tree 80,0.694417,0.692487,0.694417,0.692996
n_tree 100,0.694417,0.692101,0.694417,0.692716


### k-Means

In [6]:
iteration = 10
Results_km = []

for i in tqdm(range(iteration), desc='Progress'):
    Results_km.append(DA.kmeans(x_embedding, y))
    
Results_km = pd.DataFrame(Results_km,columns = ['ARI', 'FMI','NMI'])
Results_km

Progress: 100%|█████████████████████████████████| 10/10 [00:04<00:00,  2.24it/s]


,ARI,FMI,NMI
0,0.438129,0.578696,0.496462
1,0.438057,0.578460,0.496372
2,0.438837,0.580346,0.497236
3,0.438767,0.579693,0.497105
4,0.438155,0.578730,0.496487
5,0.438078,0.578969,0.496430
6,0.438114,0.578684,0.496448
7,0.437851,0.578622,0.496221
8,0.437841,0.577588,0.496143
9,0.437906,0.578468,0.496230


### Agglomerative 

In [7]:
idx = ['ARI', 'FMI','NMI']
AglScore = pd.DataFrame(DA.Agglomerative(x_embedding, y), index = idx)    #seurat_clusters, Cluster, celltype.l2
AglScore.T

,ARI,FMI,NMI
0,0.342313,0.509831,0.412525
